In [1]:

''' Rewrite the basic sliders example as a standalone document with JavaScript callbacks.
Present an interactive function explorer with slider widgets.
Scrub the sliders to change the properties of the ``sin`` curve, or
type into the title text box to update the title of the plot.
Run example with python sliders-standalone-code.py
In this version, the callbacks are written as strings of JS code.
'''
import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource, Band
from bokeh.models.callbacks import CustomJS
from bokeh.models.widgets import Slider, TextInput, Button
from bokeh.plotting import figure, output_notebook, output_file, show

output_notebook()
output_file("sliders-standalone.html")
# Set up data
N = 100
Re = 6378 # km
max_sma = 50000 #km
x = np.zeros(N)
y = np.zeros(N)
source = ColumnDataSource(data=dict(x=x, y=y))

t = np.linspace(0, np.pi, 10)
earth = ColumnDataSource(data=dict(x=Re*np.cos(t), upper=Re*np.sin(t), lower=-Re*np.sin(t)))

# Set up plot
plot = figure(plot_height=400, plot_width=400, title="my orbit",
              x_range=[-max_sma, max_sma], y_range=[-max_sma, max_sma], toolbar_location=None)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
plot.add_layout(Band(base='x', lower='lower', upper='upper', source=earth, 
            level='underlay', fill_alpha=0.5, line_width=1, line_color='grey', fill_color='grey'))

# Set up widgets
text = TextInput(title="title", value='my orbit')
sma = Slider(title="semi-major axis", value=42164, start=0, end=max_sma, step=10)
eccentricity = Slider(title="eccentricity", value=0.7, start=0, end=1, step=0.01)
aop = Slider(title="argument of perigee", value=0, start=0, end=360, step=0.01)

# Set up callbacks
text_callback_code ="""
    plot.title.text = text.value
"""

text.js_on_change('value', CustomJS(args=dict(text=text,plot=plot), code=text_callback_code))

sliders_callback_code="""
    console.log("coucou from callback")
    const a = sma.value,
        e = eccentricity.value,
        N = 100,
        b = a * window.Math.sqrt(1 - e**2),
        c = window.Math.sqrt(a**2 - b**2),
        omega = aop.value * window.Math.PI / 180,
        x=source.data['x'],
        y=source.data['y']
    console.log("coucou after const")
    for(let i=0; i < N; i++) {
        let t = i/(N-1) * 2 * window.Math.PI
        x[i] = a * window.Math.cos(omega) * window.Math.cos(t) - b * window.Math.sin(omega) * window.Math.sin(t) + c * window.Math.cos(omega)
        y[i] = a * window.Math.sin(omega) * window.Math.cos(t) + b * window.Math.cos(omega) * window.Math.sin(t) + c * window.Math.sin(omega)
    }
    console.log("coucou after loop")
    source.change.emit()
"""

slider_args = dict(source=source, sma=sma, eccentricity=eccentricity, aop=aop)

for w in [sma, eccentricity, aop]:
    w.js_on_change('value', CustomJS(args=slider_args, code=sliders_callback_code))

# Set up layouts and add to document
inputs = widgetbox(text, sma, eccentricity, aop)

show(row(inputs, plot, width=800))

Loading BokehJS ...